# Sequence to Sequence 

- 자연어 처리에서 기존의 일반적인 판별 모델(NLU)은 문장 내 각 단어들을 각 Node가 처리해 하나의 결과를 도출
- Sequence to Sequence는 입력된 Sequence를 이용해 하나의 나열된 Sequence를 출력하는 구조를 만들어 학습을 수행
- Seq2Seq를 이용하면, 자연어 생성 모델 구성이 가능 (기계번역 / 챗봇 / 문장요약 / STT ...)

![image1](https://wikidocs.net/images/page/24996/%EC%9D%B8%EC%BD%94%EB%8D%94%EB%94%94%EC%BD%94%EB%8D%94%EB%AA%A8%EB%8D%B8.PNG)

- Seq2Seq는 LSTM기반의 신경망 알고리즘 구조로 크게 2가지 Part
    - 1. Encoder : Input Sequence를 받아서 문장을 특정 Vector로 변환
    - 2. Decoder : 새로운 Input Sequence(번역될 문장)를 받아서 Encoder에서 처리된 Contect Vector를 이용해 Output Sequence 생성
    - Context : Encoder에 의해 변환된 Vector
    - Encoder의 Input과 Decoder의 Input을 Context Vector로 잘 매칭(처리)하여 적절한 Output 도출

- 작동 순서
  - 1. 입력 문장을 토큰화를 통해 단어(또는 형태소 단위로 처리)
  - 2. 단어 토큰을 각각의 RNN계열의 Node에 입력
  - 3. Encoder 마지막에 계산된 Node의 정보를 Context Vector로 Decoder에 전달
  - 4. Decoder는 앞서 입력받은 Context Vector를 첫 Node에 받아서 처리
  - 5. Decoder에 문장의 첫번째를 알리는 초기 토큰 (Start of Sequence, SOS)을 첫 Node에 입력
  - 6. 이후, Decoder에서 문장을 이어받아 처리하면서, 다음 등장할 확률이 높은 단어를 예측해 Output
  - 7. EOS (End of Sequence)값을 Input Sequence 끝에 넣어, 문장의 마지막을 학습
  - 8. Output 값이 생성 될 때, EOS 토큰이 등장하거나 사용자가 지정한 문장의 최대 길이만큼 Output값을 출력 후 생성을 종료

In [1]:
from keras.models import Model
from keras.layers import Input 
from keras.layers import LSTM 
from keras.layers import Dense
import numpy as np 

In [2]:
data_path = 'kor.txt'
with open(data_path, encoding='UTF-8') as file :
    lines = file.read().split('\n') # 데이터 파일을 불러올 때, 띄어쓰기(\n)를 기준으로 구분 

In [20]:
# Encoder와 Decoder에 입력 할 데이터의 구조를 생성
# 각 문장을 넣을 리스트 구조를 선언 
input_text_list  = [ ] # 영어 문장을 리스트로 선언 
target_text_list = [ ] # 한국어 문장을 리스트로 선언 
input_characters  = set() # 영어 단어 리스트 (중복 단어가 제거된 단어 사전)
target_characters = set() # 한국어 단어 리스트 ( 중복 단어가 제거된 단어 사전) 

In [9]:
num_samples = len(lines)

In [31]:
num_samples = len(lines) # 처리할 문장의 수를 변수로 선언 

for line in lines[:num_samples-1]:
    # 문장 처리 단계에서 오류 방지를 위한 Try / Except 구문을 활용 
    try: 
        # Tab Key(\t)를 기준으로 각 문장을 3개의 토큰으로 분할 
        input_text, target_text, _ = line.split('\t') # 3개로 분할 된 각 토큰을 변수로 선언 
        # Decoder에 들어갈 Input Seq (Target Text)에 SOS 토큰과 EOS 토큰을 부착 
        # Tab Key (SOS)로 Enter Key(EOS) 로 지정하여 부착 
        target_text = '\t' + target_text + '\n' 
        # 각각 나눠진 Text를 각 리스트에 추가 
        input_text_list.append(input_text)
        target_text_list.append(target_text)
    except ValueError: # 문장 처리 중 오류가 발생 했을 시 
        print('Skip Line ', line) # 어떤 문장이 생략됐는지 확인 
        continue # 이어서 진행 
    # 분할 된 모든 문장에서 고유 단어만 뽑아 리스트로 선언 (사전 만들기 위한 작업)
    for char in input_text:
        input_characters.add(char) # input_text의 단어를 가져와 input_characters에 추가
    for char in target_text:
        target_characters.add(char)

In [47]:
# 문자 사전을 만들기 위한 리스트 변환 
input_char_list = sorted(list(input_characters)) # 영어 문자 토큰을 리스트로 변환후 정렬 
target_char_list = sorted(list(target_characters)) # 한국어 문자 토큰을 리스트로 변환후 정렬 

# 전체 단어 고유 수 만큼 숫자를 부여하여, 문자 -> 숫자 사전을 구축 
num_encoder_token = len(input_char_list) # encoder에 들어갈 영어 문자 수 
num_decoder_token = len(target_char_list) # decoder 들어갈 한국어 문자 수 
# 영어 문자를 숫자로 변환시킬 Dictionary 구성 
input_token_index = dict([(char, i) for i, char in enumerate(input_char_list)])
# 한국어 문자를 숫자로 변환시킬 Dictionary 구성 
target_token_index = dict([(char, i) for i, char in enumerate(target_char_list)])

In [51]:
# 전체 문장들 중 최대 길이를 갖는 문장을 확인 (Padding을 수행 하기 위한 목적)  
max_encoder_seq_length = max([len(x) for x in input_text_list])
max_decoder_seq_length = max([len(x) for x in target_text_list])

In [53]:
# 각 문장을 변환 된 정수로 집어넣을 Matrix를 구성 
encoder_input_data = np.zeros( 
    (len(input_text_list), max_encoder_seq_length,  num_encoder_token), dtype='float32')
decoder_input_data = np.zeros(
    (len(target_text_list), max_decoder_seq_length, num_decoder_token), dtype='float32')
decoder_target_data = np.zeros(
    (len(target_text_list), max_decoder_seq_length, num_decoder_token), dtype='float32')

In [57]:
# One Hot Encoding 
# 구성된 Matrix에 각 단어에 매칭되는 숫자를 입력 
# 영어 문장과 한국어 문장을 각각 가져와 동시에 숫자를 Matrix에 입력 
for i, (input_text, target_text) in enumerate(zip(input_text_list, target_text_list)):
    # 영어 문장에 해당하는 단어를 숫자로 변환하여 Matrix에 추가 
    for t , char in enumerate(input_text):
       encoder_input_data[i ,t ,input_token_index[char]] = 1
    # 한국어 문장에 해당하는 단어를 숫자로 변환하여 Matrix에 추가 
    for t , char in enumerate(target_text):
        decoder_input_data[i ,t ,target_token_index[char]] = 1 
        if t > 0 :
            decoder_target_data[i , t-1 ,target_token_index[char]] = 1 

In [62]:
# 학습 시간과 컴퓨터 성능 문제를 고려해 데이터를 간소화 하여 학습을 진행 
batch_size = 32 # 훈련 데이터 셋 배치 크기 (64 이상)
epochs   = 20 # 훈련 에폭 수 (500회 이상) 
node_num = 64 # 각 Layer에 들어갈 Node 수 (1024개 이상) 

In [63]:
# 인코더 구성 
# 영어 문장이 들어가 LSTM 모델에 의해 학습이 수행 
encoder_input = Input(shape=(None, num_encoder_token))
encoder = LSTM(node_num, return_state=True)
# state_h : 마지막 Layer의 State / state_c 마지막 Cell State 
encoder_output, state_h, state_c = encoder(encoder_input)
# 인코더가 입력 시퀀스를 처리한 후, 얻은 정보를  encoder_state에 선언
# 디코더가 이를 기반으로 출력 시퀀스를 생성 (Context Vector) 
encoder_state = [state_h, state_c]

In [65]:
# 디코더 구성
decoder_input = Input(shape=(None, num_decoder_token))
# return_sequences=True : 각 단계에서 Seq의 전체 출력을 계산하도록 설정 
decoder_lstm  = LSTM(node_num, return_sequences=True, return_state=True)
# 앞서 처리된 Encoder의 정보를 불러와 Decoder의 초기 상태로 사용 
decoder_output,  _ , _  = decoder_lstm(decoder_input, initial_state=encoder_state )
# Softmax의 확률 분포 값을 이용하여 출력값의 확률 값을 계산 
decoder_dense  = Dense(num_decoder_token, activation='softmax')
decoder_output = decoder_dense(decoder_output)

# Seq2Seq 모델을 정의 
# 두개의 Input이 들어가, 하나의 Output 나오는 구조
model = Model( [encoder_input, decoder_input] , decoder_output)

In [66]:
# 학습 실시 
model.compile(optimizer='adam', loss='categorical_crossentropy')
model.fit( [encoder_input_data, decoder_input_data], decoder_target_data,
           batch_size=batch_size, epochs=epochs, validation_split=0.3) 

Epoch 1/20
387/387 ━━━━━━━━━━━━━━━━━━━━ 229s 561ms/step - loss: 0.3195 - val_loss: 0.3291
Epoch 2/20
387/387 ━━━━━━━━━━━━━━━━━━━━ 156s 401ms/step - loss: 0.3023 - val_loss: 0.3185
Epoch 3/20
387/387 ━━━━━━━━━━━━━━━━━━━━ 163s 417ms/step - loss: 0.2894 - val_loss: 0.3017
Epoch 4/20
387/387 ━━━━━━━━━━━━━━━━━━━━ 162s 412ms/step - loss: 0.2747 - val_loss: 0.2915
Epoch 5/20
387/387 ━━━━━━━━━━━━━━━━━━━━ 134s 342ms/step - loss: 0.2672 - val_loss: 0.2858
Epoch 6/20
387/387 ━━━━━━━━━━━━━━━━━━━━ 152s 387ms/step - loss: 0.2628 - val_loss: 0.2837
Epoch 7/20
387/387 ━━━━━━━━━━━━━━━━━━━━ 161s 412ms/step - loss: 0.2607 - val_loss: 0.2820
Epoch 8/20
387/387 ━━━━━━━━━━━━━━━━━━━━ 158s 403ms/step - loss: 0.2591 - val_loss: 0.2802
Epoch 9/20
387/387 ━━━━━━━━━━━━━━━━━━━━ 152s 387ms/step - loss: 0.2578 - val_loss: 0.2795
Epoch 10/20
387/387 ━━━━━━━━━━━━━━━━━━━━ 152s 388ms/step - loss: 0.2577 - val_loss: 0.2792
Epoch 11/20
387/387 ━━━━━━━━━━━━━━━━━━━━ 147s 375ms/step - loss: 0.2575 - val_loss: 0.2781
Epoch 12

In [67]:
# 학습 이후(NLU) 추론 및 생성(NLG)모델을 구성 
# 학습 과정에서 구성한 인코더의 입력과 인코더의 상태를 기반으로 새로운 모델을 생성 
encoder_model = Model(encoder_input, encoder_state)
# 디코더의 LSTM 레이어에 전달될 초기 상태를 정의하는 입력 Layer를 구성 
decoder_state_input_h = Input(shape=(node_num, )) # Layer State를 초기화 
decoder_state_input_c = Input(shape=(node_num, )) # Cell State를 초기화 
# 초기 Decoder의 Input Node를 구성 
decoder_state_input = [decoder_state_input_h, decoder_state_input_c]
# 구성된 Input Node를 이용해, Decoder의 Layer를 구성 
decoder_output, state_h, state_c = decoder_lstm(
                                decoder_input, initial_state= decoder_state_input)
# 다음 Step으로 전달 될 상태를 지정 
decoder_state = [state_h, state_c]
# 확률 분포를 계산하는 Decoder Output에 전달 하여 Update 
decoder_ouput_proba = decoder_dense(decoder_output)
# 다음 단계의 토큰 (단어, 문자)를 예측하고 다음 문장을 생성하기 위한 생태를 생성 
decoder_model= Model([decoder_input]+ decoder_state_input, 
                     [decoder_output]+decoder_state)
# Decoder가 계산 결과 이용해, 역매핑(숫자->문자)변환 작업을 수행 
reverse_input_char_index = dict( (i, char) for char, i in input_token_index.items() )
reverse_target_char_index = dict( (i, char) for char, i in target_token_index.items() )

In [70]:
# 입력 문장이 들어오면, 해당 문장을 번역하는 함수를 구성 
def decode_sequence(input_seq):
    # 입력 받은 Sequence를 이용해, Encoder에 넣어 Context Vector를 예측 
    state_value = encoder_model.predict(input_seq, verbose=0)
    
    # Decoder의 초기 Sequence를 결정 
    target_Seq = np.zeros((1,1,num_decoder_token))
    # 시작토큰 (SOS)의 위치를 1로 설정해서, Decoder가 예측을 시작할 부분을 지정 
    target_Seq[0,0, target_token_index['\t']] = 1 

    # 디코딩이 종료가 될 시점의 변수를 지정 
    # 문장 끝에 EOS가 오거나 또는 사용자가 지정한 문장 최대 길이에 도달하면 생성이 중지 
    stop_condition = True # 종료조건이 오면, 해당 변수를 False 
    decoder_sentence = '' # 생성될 문자열이 들어갈 변수 
    while stop_condition :
        # 디코더의 Layer 정보를 이용해서 Ouput Token을 출력 
        output_token, h, c = decoder_model.predict(
                                        [target_Seq] + state_value, verbose=0)
        # 가장 확률이 높게 계산된 숫자(단어의 Index)를 변수로 선언 
        sampled_token_index = np.argmax(output_token[0, -1 , : ])
        # 해당 Index를 문자로 변환 
        sampled_char = reverse_target_char_index[sampled_token_index]
        # 변환된 문자를 decoder_sentence에 추가 
        decoder_sentence += sampled_char

        # EOS 나 최대 문장 길이에 도달할 경우, Stop Condition을 False 바꿔 반복을 종료 
        if (sampled_char=='\n' or len(decoder_sentence) > max_decoder_seq_length):
            stop_condition = False 
        # 종료가 되지 않을 시, 다음 단어(문자)를 예측하기 위해 초기화 
        target_Seq = np.zeros((1,1,num_decoder_token))
        target_Seq[0,0, sampled_token_index] = 1 
        state_value = [h,c]
    # 종료조건에 의해 반복문이 멈추면 생성된 문장을 출력 
    return decoder_sentence

In [71]:
# 새로운 문장을 직접 입력해 번역 실행 
def preprocess_input_sequence(setence):
    input_seq = np.zeros((1, max_encoder_seq_length, num_encoder_token), dtype='float32')
    for i, char in enumerate(setence):
        # 입력 문장의 각 문자에 대해 One Hot Encoding 수행 
        input_seq[0, i, input_token_index[char]] = 1 
    return input_seq # Text to Sequence 

In [72]:
# 변환된 Sequence를 모델에 넣어 번역 문자 생성 
def translate(input_text):
    # 입력문장 전처리
    input_seq = preprocess_input_sequence(input_text)
    # 번역을 수행 
    decode_sentence = decode_sequence(input_seq)
    return decode_sentence

In [73]:
# 구성된 함수에 입력문장을 넣기 
input_sentence = "Show me the money!"
translate(input_sentence)

'갔갔갔갔갔aaaaBB0000    ,  ,,    "",   ,  ,  ",   /",   , "    ",  " "  /",    ""   ,  ",    ",  " ,""    "  " " "/,    /"      "/  "  ,/""   " ,,     ""   "  ",   /  ""   /  //,   """"    " ""   , ",  ,  "  /,  ,,  "     ""       /,  "  " ""   / " """"   ,  " ,"  "/  "   /     /,,    ,""    ,  ""  ,,  '

In [74]:
input_sentence = "This is pen."
translate(input_sentence)

'갔갔갔갔갔aaaaBB0000    ,  ,,    "",   ,  ,  ",   /",   , "    ",  " "  /",    ""   ,  ",    ",  " ,""    "  " " "/,    /"      "/  "  ,/""   " ,,     ""   "  ",   /  ""   /  //,   """"    " ""   , ",  ,  "  /,  ,,  "     ""       /,  "  " ""   / " """"   ,  " ,"  "/  "   /     /,,    ,""    ,  ""  ,,  '